<a href="https://colab.research.google.com/github/crodier1/data_science/blob/main/Bitcoin_News_Word_Association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apyori
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=a149486dfb9d59ac4b66500aef1f639bcfd19d5c945c46fc35da0502e25a87bf
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 235 kB 32.2 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
     |████████████████████████████████| 4.4 MB 61.9 MB/s 
     |████████████████████████████████| 1.2 MB 46.1 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
     |████████████████████████████████| 596 kB 57.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
  Created wheel for keybert: filename=keybert-0.5.1-py3-none

In [ ]:
import pandas as pd
from apyori import apriori
from keybert import KeyBERT

In [ ]:
!pip install newsapi-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datetime import date, timedelta

today = date.today()

d = timedelta(days = 14)
two_weeks_ago = today - d

In [ ]:
# documentation
# https://newsapi.org/docs/client-libraries/python
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='20c0c0a0b2bf400fa987f45a9ded6718')


all_articles = newsapi.get_everything(q='bitcoin',                                      
                                      from_param=str(two_weeks_ago),
                                      to=today,
                                      language='en',
                                      sort_by='relevancy'
                                      )

all_articles = all_articles['articles']

In [ ]:
kw_model = KeyBERT()

def get_transactions(article):  
  return [t[0] for t in kw_model.extract_keywords(article)]

In [ ]:
transactions = [ get_transactions(feeds['content']) for feeds in  all_articles]

In [ ]:
times_word_appears = 3

number_transactions = len(transactions)

min_support = times_word_appears / number_transactions

min_confidence = 0.2

min_lift = 3

rules = apriori(transactions= transactions, min_support = min_support,  min_confidence = min_confidence, min_lift= min_lift, min_length = 2, max_length = 2)

In [ ]:
results = list(rules)

In [ ]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [ ]:
resultsinDataFrame.head(10)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,bankruptcy,celsius,0.03,1.000000,25.000000
1,celsius,crypto,0.03,0.750000,3.947368
2,closed,stocks,0.03,1.000000,9.090909
3,earnings,stocks,0.04,0.571429,5.194805
4,earnings,tesla,0.06,0.857143,6.122449
5,elon,musk,0.03,1.000000,25.000000
6,holdings,musk,0.03,0.600000,15.000000
7,holdings,tesla,0.03,0.600000,4.285714
8,investors,stocks,0.04,0.571429,5.194805
9,mining,texas,0.03,0.500000,16.666667
